In [2]:
# import all libraries
import quandl
import pandas as pd
import numpy as np
from scipy.linalg import cholesky
from pandas.tools.plotting import scatter_matrix
import seaborn as sns
import math

# import data from quandl
hdfc= quandl.get("NSE/HDFC", authtoken="uLQWa1ux8ZUKptMjSzG5", collapse="quarterly", start_date="2002-01-01", end_date="2016-12-31")

reliance =quandl.get("NSE/RELIANCE", authtoken="uLQWa1ux8ZUKptMjSzG5", collapse="quarterly", start_date="2002-01-01", end_date="2016-12-31")

ongc= quandl.get("NSE/ONGC", authtoken="uLQWa1ux8ZUKptMjSzG5", collapse="quarterly", start_date="2002-01-01", end_date="2016-12-31")

# extracting close price as dataframes
hdfc=pd.DataFrame(hdfc,columns=['Close'])
reliance=pd.DataFrame(reliance,columns=['Close'])
ongc=pd.DataFrame(ongc,columns=['Close'])

# concatinate all dataframes in to a single DF
data = pd.concat([hdfc,reliance,ongc], axis=1)

# calculating log returns
# log_returns - used to find mu and sigma for geometric_brownian_motion
log_returns=np.log(data/data.shift()).dropna()

# finding the covariance
covariance_matrix=log_returns.cov()

# set up the parameters for the simulation
n_assets = 3
#  data downloded as quarterly 
dt = 1./4
# calculating mu and sigma
mu = log_returns.mean().values
print("mu",mu)
sigma = log_returns.std().values  
no_years_sim=30
no_quarters=4
# initialize price matrix

def convert_to_prices(brownian_motion):
#     method used for calculating prices
    price_matrix = np.full((row+1,column),100)
    for i in range(column):
        for j in range(row):
            
            price_matrix[j+1,i]=(brownian_motion[j,i])*price_matrix[j,i]
    return price_matrix


def brownian_motion():
#     finding the multivariate_normal
    Wiener_process = np.random.multivariate_normal(mu, covariance_matrix,(no_years_sim*no_quarters))
#     dimension(row and column) of Wiener_process matrix
    shap=Wiener_process.shape
    global row
    global column
    row=shap[0]
    column=shap[1]
    return Wiener_process


def geometric_brownian_motion():
    #here we will calculate geometric brownian motion process using equation exp((mu-0.5*sigma**2)*dt+(sigma*wt))
    #first part 'sigma_pow_mu_delta' 
    #second part is the  Wiener_process for that we call another method  brownian_motion_log_returns()   
    sigma_pow_mu_delta =(mu-0.5*sigma**2)*dt
    Wiener_process = brownian_motion()
    geometric_brownian_motion=sigma_pow_mu_delta+Wiener_process
    geometric_brownian_motion=np.exp(geometric_brownian_motion)
    return geometric_brownian_motion


def geometric_brownian_motion_levels():
        #Returns a sequence of price levels for an asset which evolves according to a geometric brownian motion
        #return: the price levels for the asset
    return convert_to_prices(geometric_brownian_motion())

# PLOTTING
# parameters for graph
x_axis=[]
for i in range(len(data)):
    x_axis.append(i)
    
import plotly.graph_objs as go
from plotly.offline import plot

data=[]

for j in range(100):
#   start-->function call
    price_matrix=geometric_brownian_motion_levels()
    price_data = pd.DataFrame(price_matrix,columns=['hdfc','reliance','ongc'])
    print(price_data)
    print("len(price_data)",len(price_data))
    price_data=price_data.T.as_matrix()
#     dimension of price_data
    shapee=price_data.shape
    row_price_data=shapee[0]

#     parameters for ploting graph
#     colors 
    c=['rgb(22, 96, 167)','rgb(205, 12, 24)','rgb(102, 153, 0)']
#     name of assets
    text=['hdfc','reliance','ongc']
    name=['hdfc','reliance','ongc']
    for i in range(0,row_price_data):
        name=['hdfc','reliance','ongc']
        trace=go.Scatter(
        x=x_axis,
        y=price_data[i],
        text=text[i],
        name=name[i],
        marker=dict(size=12,
                
                color=c[i]
               )
        )
        data.append(trace)
plot(data, filename='portfolio_graph.html')


('mu', array([ 0.01037818,  0.02173143, -0.00612531]))


ImportError: No module named plotly.graph_objs